In [6]:
!pip install seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# Load dataset
data = pd.read_csv("sentimentdataset.csv")  # Columns: 'Text', 'Sentiment'

# Encode labels if not numeric
label_encoder = LabelEncoder()
data['Sentiment'] = label_encoder.fit_transform(data['Sentiment'])  # 0/1 or 0/1/2

data.head()


In [29]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    data['Text'], data['Sentiment'], test_size=0.2, random_state=42
)

# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding
max_len = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)


In [30]:
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=64, input_length=max_len))
model.add(SimpleRNN(64, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))  # Use 'softmax' if more than 2 classes

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


C:\Users\Owner\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [31]:
history = model.fit(X_train_pad, y_train, epochs=5, validation_data=(X_test_pad, y_test))


Epoch 1/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 7s 73ms/step - accuracy: 0.0096 - loss: -111.4631 - val_accuracy: 0.0000e+00 - val_loss: -311.1048
Epoch 2/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.0090 - loss: -404.7414 - val_accuracy: 0.0000e+00 - val_loss: -124.8330
Epoch 3/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.0078 - loss: -385.8876 - val_accuracy: 0.0000e+00 - val_loss: -513.1946
Epoch 4/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.0082 - loss: -696.5084 - val_accuracy: 0.0000e+00 - val_loss: -764.5587
Epoch 5/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.0055 - loss: -902.3528 - val_accuracy: 0.0000e+00 - val_loss: -949.8784


In [32]:
# Evaluate
loss, accuracy = model.evaluate(X_test_pad, y_test)
print("Test Accuracy:", accuracy)

# Predict new text
text_to_analyze = ["I love the way you handled the service. Just perfect!"]
seq = tokenizer.texts_to_sequences(text_to_analyze)
padded = pad_sequences(seq, maxlen=max_len)
pred = model.predict(padded)
print("Predicted Sentiment:", "Positive" if pred[0][0] > 0.5 else "Negative")


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0000e+00 - loss: -984.6109
Test Accuracy: 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step
Predicted Sentiment: Positive
